In [1]:
import numpy as np
import os
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [2]:
def get_images_and_labels(path):
 image_paths = [os.path.join(path, f) for f in os.listdir(path)]
 images = []
 labels = []
 for image_path in image_paths:
        image_pil = Image.open(image_path).convert('L')
        image = np.array(image_pil, 'uint8')
        nbr = int(os.path.split(image_path)[1].split(".")[0].replace("subject", ""))
        labels.append(nbr)
        images.append(image)
 return images, labels

def eigen_decompose(data,num_PCA_components):
    inp_dim=data.shape[1]
    num_train=data.shape[0]
    #principal component analysis
    mean_face, eigen_vec=cv2.PCACompute(data, mean=None, maxComponents=num_PCA_components)
    return mean_face, eigen_vec

def feature_transform(data,eigens,mean):
    num_train=data.shape[0]
    inp_dim=data.shape[1]
    m=np.zeros([num_train, inp_dim])
    for k in range(num_train):
        m[k,:]=mean
    d=np.transpose(data-mean)
    features=np.dot(eigens, d)
    f=np.transpose(features)
    return f



In [3]:
if __name__ == '__main__':
    #loading the images
    path ="C:\\Users\\aarus\\Downloads\\ELL784\\Assignments\\Assignment2\\yalefaces"
    images_read, labels = get_images_and_labels(path)
    labels=np.array(labels)
    #creating data matrix
    num_images=164
    height=243
    width=320
    input_dimension=height*width
    data_matrix=np.zeros([num_images, input_dimension])
    for n_idx in range(num_images):
        data_matrix[n_idx,:]=images_read[n_idx].flatten()
    #training and test sets
    data_train, data_test, label_train, label_test = train_test_split(data_matrix, labels, test_size=0.1)
    num_test=data_test.shape[0]
    #eigenfaces using PCA
    num_PCA_components=50
    mean, eigens=eigen_decompose(data_train,num_PCA_components)
   
    
    #feature transformation
    feature_train=feature_transform(data_train,eigens,mean)
    #training the SVM classifier with linaer kernel
    svm_model_linear = SVC(kernel = 'linear', C = 1).fit(feature_train, label_train)
    #testing
    feature_test=feature_transform(data_test,eigens,mean)
    svm_predictions = svm_model_linear.predict(feature_test)
    accuracy = svm_model_linear.score(feature_test, label_test)
    print(100*accuracy)
    #mean and eigenfaces
    mean_face=np.reshape(mean, [height,width])
    mean_img=Image.fromarray(mean_face)
    mean_img.show()
    #cv2.imwrite("mean face eigen.jpg", mean_face)
    #for k in range(num_PCA_components):
    #    filename="Eigenface"+str(k+1)+".jpg"
    #    eigen_face=np.reshape(eigens[k,:]+mean, [height,width])
    #    cv2.imwrite(filename, eigen_face)
    print("DONE...........")


100.0
DONE...........
